# 젯슨나노에서 YOLOv5 실행하기(작성중)

작성중입니다. 헛다리 짚고 있습니다ㅜ..
CSI 카메라 이미지를 쓰는 방법을

## 1. 터미널에서 pip3를 설치합니다.

```bash
sudo apt update
sudo apt upgrade
sudo apt install -y python3-pip
pip3 install --upgrade pip
```

## 2. 깃헙의 yolov5 레포지토리를 클론합니다.

```bash
git clone https://github.com/ultralytics/yolov5
```

## 3. 종속패키지를 모두 설치합니다.

단, torch와 torchvision이 버전이 맞지 않으므로

requirements.txt에서 torch 및 torchvision 라인을 주석처리합니다.

(torch와 torchvision은 별도로 버전에 맞게 다운로드할 예정입니다.)

```bash
cd yolov5
vim requirements.txt
```

주석처리를 완료하였으면 저장하고 종료한 후(:wq)

아래 명령어를 실행합니다.

```bash
sudo apt install -y libfreetype6-dev
pip3 install -r requirements.txt
```

## 4. 버전에 따라 torch와 torchvision을 다운받습니다.

- JetPack 4.4~4.6 에서는 Python3.6에 아래 파일을 사용하여 설치해야 합니다.
https://nvidia.box.com/shared/static/fjtbno0vpo676a25cgvuqc1wty0fkkg6.whl

- JetPack 5.0 이상에서는 Python3.8에 아래 파일을 사용하여 설치합니다.
https://developer.download.nvidia.com/compute/redist/jp/v50/pytorch/torch-1.12.0a0+2c916ef.nv22.3-cp38-cp38-linux_aarch64.whl

위 링크를 클릭하여 다운로드 후 설치해도 되지만
wget을 통해 설치 가능합니다.
(아래 코드를 실행하세요.)

- JetPack 4.4~4.6

```bash
sudo apt-get install -y libopenblas-base libopenmpi-dev
wget https://nvidia.box.com/shared/static/fjtbno0vpo676a25cgvuqc1wty0fkkg6.whl -O torch-1.10.0-cp36-cp36m-linux_aarch64.whl
pip3 install torch-1.10.0-cp36-cp36m-linux_aarch64.whl
```

- JetPack 5.0~

```bash
sudo apt-get install -y libopenblas-base libopenmpi-dev
wget https://developer.download.nvidia.com/compute/redist/jp/v50/pytorch/torch-1.12.0a0+2c916ef.nv22.3-cp38-cp38-linux_aarch64.whl
pip3 install torch-1.12.0a0+2c916ef.nv22.3-cp38-cp38-linux_aarch64.whl
```

정상적으로 torch가 설치되었다면

이어서 torchvision을 설치합니다. (torch의 버전에 맞게 설치해야 합니다.)

- JetPack 4.4~4.6

```bash
sudo apt install -y libjpeg-dev zlib1g-dev
git clone --branch v0.11.1 https://github.com/pytorch/vision torchvision
cd torchvision
sudo python3 setup.py install
```

- JetPack 5.0~

```bash
sudo apt install -y libjpeg-dev zlib1g-dev
git clone --branch v0.13.0 https://github.com/pytorch/vision torchvision
cd torchvision
sudo python3 setup.py install
```

## 5. 스트리밍을 위한 라이브러리 중 deepstream을 설치하겠습니다.

깃헙에서 deepstream을 클론합니다.

```bash
cd ~
git clone https://github.com/marcoslucianops/DeepStream-Yolo
```

클론이 완료되었으면 deepstream 내부의 export_yoloV5.py를 ~/yolov5에 복사합니다.

```bash
cp DeepStream-Yolo/utils/export_yoloV5.py yolov5
```

이제 yolo모델 pt파일을 다운받습니다. (s모델 6.1 버전)

```bash
cd yolov5
wget https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5s.pt
```

다운이 완료되었으면 cfg파일 및 wts파일을 생성합니다.
만약 추론 해상도를 640(default)에서 1280으로 변경하고 싶다면 뒤에 `-s 1280`을 추가합니다.
```bash
python3 export_yoloV5.py -w yolov5s.pt
```

생성된 cfg와 wts 파일을 deepstream 폴더에 넣습니다.

```bash
cp yolov5s.cfg ~/DeepStream-Yolo
cp yolov5s.wts ~/DeepStream-Yolo
```

복사를 마쳤으면 CUDA 버전에 맞게 라이브러리를 컴파일합니다.

```bash
cd ~/DeepStream-Yolo
CUDA_VER=11.4 make -C nvdsinfer_custom_impl_Yolo  # for DeepStream 6.1
CUDA_VER=10.2 make -C nvdsinfer_custom_impl_Yolo  # for DeepStream 6.0.1 / 6.0
```

컴파일이 정상적으로 종료되었으면, 

config_infer_primary_yoloV5.txt와 deepstream_app_config을 수정합니다.

- config_infer_primary_yoloV5.txt

```
[property]
...
custom-network-config=yolov5s.cfg
model-file=yolov5s.wts
...
```

저장하고 종료(:wq)

- deepstream_app_config 

```
...
[primary-gie]
...
config-file=config_infer_primary_yoloV5.txt
...
[source0]
...
uri=file:///opt/nvidia/deepstream/deepstream/samples/streams/sample_1080p_h264.mp4
```

이제 추론을 실행합니다.

```bash
deepstream-app -c deepstream_app_config.txt
```
